In [1]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [2]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
config_dir = r"C:\Users\PC\Documents\projects\carPlates\models\lettersModel\pipeline.config"
ckpt_dir = r"C:\Users\PC\Documents\projects\carPlates\models\lettersModel\ckpt-101"

In [4]:
configs = config_util.get_configs_from_pipeline_file(config_dir)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(ckpt_dir).expect_partial()

#@tf.function
def detect_fn(image):

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

In [13]:
def detect_img(img):
    image_np = np.array(img)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

    detections = detect_fn(input_tensor)

        # All outputs are batches tensors.
        # Convert to numpy arrays, and take index [0] to remove the batch dimension.
        # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
    detections['num_detections'] = num_detections

        # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    
    
    output_detections = []
    
    for i in range(len(detections['detection_boxes'])):
        if detections['detection_scores'][i] > 0.4:
            output_detections.append(("letter", detections['detection_boxes_strided'][i]))
    
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                {0 : "1"},
                use_normalized_coordinates=True,
                max_boxes_to_draw=200,
                min_score_thresh=.4,
                agnostic_mode=False,
                line_thickness = 1,
                skip_labels=True,
                skip_scores=True
        )
    return Image.fromarray(image_np_with_detections)

In [6]:
img_path = r"E:\DataSets\carPlatesDataset\raw_plates"
img_names = os.listdir(img_path)
test_path = r"E:\DataSets\carPlatesDataset\predictions\letters420"

In [7]:
train_df = pd.read_csv(r"C:\Users\PC\Documents\projects\carPlates\data\letters420\train_labels.csv")

In [8]:
img_directories = list(set(train_df['filename']))

In [9]:
img_paths = [os.path.join(img_path, img_name) for img_name in img_names]
test_paths = []
for path in img_paths:
    if path not in img_directories:
        test_paths.append(path)
    
    

In [10]:
test_paths

['E:\\DataSets\\carPlatesDataset\\raw_plates\\IMG_20220330_121656_0.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw_plates\\IMG_20220330_121656_1.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw_plates\\IMG_20220330_121658_0.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw_plates\\IMG_20220330_121658_1.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw_plates\\IMG_20220330_121658_2.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw_plates\\IMG_20220330_121658_3.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw_plates\\IMG_20220330_121704_1.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw_plates\\IMG_20220330_121704_2.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw_plates\\IMG_20220330_121704_3.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw_plates\\IMG_20220330_121704_4.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw_plates\\IMG_20220330_121706_1.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw_plates\\IMG_20220330_121706_3.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw_plates\\IMG_20220330_121706_4.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw_

In [11]:
len(test_paths)

1767

In [14]:
i = 0
for directory in test_paths:
    img = Image.open(directory)
    detection = detect_img(img)
    detection.save(os.path.join(r"E:\DataSets\carPlatesDataset\predictions\letters420", f"{i}" + ".jpg"))
    print(i)
    i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [50]:
path = os.path.join(img_path, test_paths[0])
img = Image.open(path)
dsk_img = Image.open(r"C:\Users\PC\OneDrive\Desktop\test4.jpeg")
detection, output_detections = detect_img(img)
detection.show()

dict_items([('detection_scores', array([0.77187395, 0.7433272 , 0.7424787 , 0.6984243 , 0.65364033,
       0.44410685, 0.04337851, 0.03691638, 0.02739033, 0.02651289,
       0.02628846, 0.02604754, 0.02370136, 0.01714181, 0.01709035,
       0.01703402, 0.0162413 , 0.01554155, 0.01313605, 0.01203198,
       0.01188164, 0.01182735, 0.01143873, 0.01072817, 0.01072604,
       0.01060795, 0.01045647, 0.01037901, 0.01025113, 0.00994098,
       0.0098303 , 0.00971948, 0.00940618, 0.00931582, 0.00915609,
       0.00903313, 0.00899688, 0.00884409, 0.00873669, 0.00849414,
       0.00847617, 0.00839291, 0.00832727, 0.00813728, 0.00807478,
       0.0080206 , 0.00800984, 0.00798728, 0.00796612, 0.00784666,
       0.00773   , 0.00766144, 0.00762659, 0.00755381, 0.00752057,
       0.00744997, 0.00744364, 0.00741958, 0.00736316, 0.00724467,
       0.007203  , 0.00718136, 0.00717871, 0.00717209, 0.00712769,
       0.00707586, 0.00705055, 0.00701988, 0.00698188, 0.00698071,
       0.00697872, 0.00689755

In [51]:
output_detections

[('letter', [36, 97, 54, 111]),
 ('letter', [36, 23, 58, 35]),
 ('letter', [35, 48, 55, 62]),
 ('letter', [39, 82, 54, 93]),
 ('letter', [36, 36, 56, 47]),
 ('letter', [35, 13, 66, 19])]

In [12]:
import cv2

In [13]:
raw_vid_paths = r"E:\DataSets\carPlatesDataset\raw_vids"
raw_vid_names = os.listdir(raw_vid_paths)

In [14]:
raw_vid = cv2.VideoCapture(os.path.join(raw_vid_paths, raw_vid_names[0]))


In [15]:
import cv2
cap = cv2.VideoCapture(0)
count = 0
while cap.isOpened():
    ret,frame = cap.read()
    #frame = detect_img(frame)
    frame = np.array(Image.fromarray(frame))
    cv2.imshow('window-name', frame)
    #cv2.imwrite("frame%d.jpg" % count, frame)
    count = count + 1
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows() # destroy all opened window

AttributeError: 'NoneType' object has no attribute '__array_interface__'

In [55]:
import os
from lxml import etree as et
def create_xml(imgfilepath, object_list, savedir):
    """
    params:
    - imgfilepath: path of corresponding img file. only the basename will be actually used.
    - object_list: python list of objects. the format of each element is up to the user, but for this example, each element will be a tuple of (classname, [x1,y1,x2,y2])
    - savedir: output directory to save generated xml file
    """
    basename = os.path.basename(imgfilepath)
    filename, _ = os.path.splitext(basename)
    
    root = et.Element('annotation')
    fn_elem = et.SubElement(root, 'filename')
    bn = os.path.basename(imgfilepath)
    fn, _ = os.path.splitext(bn)
    img_bn = f"{fn}.jpg"
    fn_elem.text = img_bn
    for classname, (y1,x1,y2,x2) in object_list:
        object_elem = et.SubElement(root, 'object')
        name = et.SubElement(object_elem, 'name')
        name.text = classname
        bndbox = et.SubElement(object_elem, 'bndbox')
        xmin = et.SubElement(bndbox, 'xmin')
        xmin.text = str(x1)
        xmax = et.SubElement(bndbox, 'xmax')
        xmax.text = str(x2)
        ymin = et.SubElement(bndbox, 'ymin')
        ymin.text = str(y1)
        ymax = et.SubElement(bndbox, 'ymax')
        ymax.text = str(y2)
    out = et.tostring(root, pretty_print=True, encoding='utf8') # if element attributes contains some non ascii characters, then need to specify encoding. if not the case, then encoding doesn't need to be set
    savepath = os.path.join(savedir, f'{fn}.xml')
    with open(savepath, 'wb') as fd:
        fd.write(out)

In [57]:
path = os.path.join(img_path, test_paths[0])
img = Image.open(path)

detection, output_detections = detect_img(img)
detection.show()

savedir = r"E:\DataSets\carPlatesDataset\newLetterBoxes"
create_xml(path, output_detections, savedir)